# Linear Quantization 1, part 1
##引言  
###1.Quantization refers to the process of mapping a large set to a smaller set of values.  
###2.有很多量化技术，这里只专注于线性量化。量化的对象是权重，也就是神经网络参数，但也可以对激活值（the activations）进行量化。  
###3.如果是对训练后的神经网络进行量化，称为后训练量化（post training quantization, PTQ）  
###4.量化的优点：  
a.模型体积更小  
b.速度提升：memory bandwidth, faster operations(GEMM:General Matrix Multiply, GEMV:Genera Matrix Vector Multiplication)  
###5.量化中的挑战：  
a.Quantization  error（量化误差）  
b.Retraining(Quantization Aware Training)（再训练（量化感知训练））  
c.Limited Hardware support(有限的硬件资源)  
d.Calibration dataset needed(需要校准数据集)  
e.packing/unpacking  

##Linear Quantization
###线性量化使用线性映射将高精度范围（例如浮点32），映射到低精度范围（如int8）。s为缩放因子，z为零点。如下图：

![LinearQuantization.jpg](./img/01-LinearQuantization.jpg)

###缩放因子以与原始张量相同的数据类型存储，z以与量化后张量相同的数据类型存储。

###s为缩放因子，z为零点。

##为了得到量化张量q，我们推导推导公式：

 ###q = int(round(r/s + z))

![getq](./img/01-getq.jpg)

###代码

In [56]:
import torch

In [57]:
def linear_q_with_scale_and_zero_point(
    tensor, scale, zero_point, dtype=torch.int8):
    scaled_and_shifted_tensor = tensor / scale + zero_point
    
    rounded_tensor = torch.round(scaled_and_shifted_tensor)
    #最后一步是，确保我们的舍入张量在最小量化值和最大量化值之间
    q_min = torch.iinfo(dtype).min#iInfo方法获取最小值和最大值
    q_max = torch.iinfo(dtype).max

    q_tensor = rounded_tensor.clamp(q_min, q_max).to(dtype)#定义量化张量（使用to()函数转换为我们想要的量化数据类型）
    return q_tensor

In [58]:
test_tensor = torch.tensor([[191.6, -13.5, 728.6],[92.14, 295.5,-184],[0,684.6, 245.5]])

In [59]:
scale = 3.5
zero_point = -70

In [60]:
quantized_tensor = linear_q_with_scale_and_zero_point(test_tensor, scale, zero_point)

In [61]:
quantized_tensor

tensor([[ -15,  -74,  127],
        [ -44,   14, -123],
        [ -70,  126,    0]], dtype=torch.int8)

###现在我们已经获取了量化张量quantized_tensor，让我们对其进行反量化，看看量化的精度如何。

In [62]:
dequantized_tensor = scale * (quantized_tensor.float() - zero_point)

In [63]:
dequantized_tensor

tensor([[ 192.5000,  -14.0000,  689.5000],
        [  91.0000,  294.0000, -185.5000],
        [   0.0000,  686.0000,  245.0000]])

In [64]:
###如果不转换成浮点数，

In [65]:
scale * (quantized_tensor - zero_point)

tensor([[ 192.5000,  -14.0000, -206.5000],
        [  91.0000,  294.0000, -185.5000],
        [   0.0000, -210.0000,  245.0000]])

In [66]:
def linear_dequantization(quantized_tensor, scale, zero_point):
    return scale * (quantized_tensor.float() - zero_point)

In [67]:
!pip install helper

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [68]:
from helper import plot_quantization_errors

grp 模块在 Windows 系统中不可用。


ModuleNotFoundError: No module named 'pwd'